<a href="https://colab.research.google.com/github/BingHungLee/Heat_Transfer/blob/master/Double_Pipe_HX_Bare_on_both_side_(rev_122419).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
Pipe_Out
  Fluid: Oil
  M_dot: 1 kg/s
  T_in: 370K
  T_out: 340K
  do = 0.0483 m
  di = 0.0409 m

Pipe_In
  Fluid: Water
  M_dot: 0.767 kg/s
  T_in: 303K
  Do = 0.089 m
  Di = 0.075 m 

Pipe Material
  C ~ 0.5%
  kw = 53 W/m.K
'''

Oil_m_dot = 1
Oil_Tin = 370
Oil_Tout = 340
Oil_Do = 0.089
Oil_Di = 0.075
Oil_visc = 0.030233
Oil_Pr = 472.5
Oil_k = 0.1375
Cpoil_avg = (2200+2080)/2 #J/kg.K

Water_m_dot = 0.767
Water_Tin = 303
Water_do = 0.0483
Water_di = 0.04094
Water_visc = 0.000654
Water_Pr = 4.33
Water_k = 0.618
Cpwater_avg = 4180

Material_kw = 53

In [0]:
import math

def Reynold(G,di,visc):
  return G*di/visc

def Across_Solid(di):
  return math.pi * math.pow(di,2)/4

def Across_Donut(Di,do):
  return math.pi/4*(math.pow(Di,2)-math.pow(do,2))  

#[Gnielinski]  
def hG(Re,Pr,k,di):
  f = math.pow(1.58*math.log(Re)-3.28,-2)
  Nu = ((f/2)*(Re-1000)*Pr)/(1.07+12.7*math.pow(f/2,0.5)*(math.pow(Pr,2/3)-1))
  return k*Nu/di

#[Taborek 1998 - 2000<Re<8000]

#[Taborek 1998 - Re<2000]

def hlam(Di,do,Re,Pr,Dh,L,k):
  Nu = 3.66+1.2*math.pow(Di/do,-0.8)+(1+0.14*math.pow(Di/do,-0.5))*((0.19*math.pow(Re*Pr*Dh/L,0.8))/(1.07+0.117*math.pow(Re*Pr*Dh/L,0.467)))
  return k*Nu/Dh

def LMTD(Thi,Tho,Tci,Tco):
  T1 = Tho-Tci
  T2 = Thi-Tco
  return (T1-T2)/math.log(T1/T2)

def Uvalue(ni,hi,di,no,ho,do,kw):
  return math.pow(do/(ni*hi*di)+1/(no*ho)+do*math.log(do/di)/(2*kw),-1)
  

In [39]:
Qtotal = Oil_m_dot * Cpoil_avg * (Oil_Tin-Oil_Tout)
Water_Tout = Water_Tin + Qtotal/(Water_m_dot*Cpwater_avg)

#Pipe Inside
Aci=Across_Solid(Water_di)
Rei=Reynold(Water_m_dot/Aci,Water_di,Water_visc)
hi = hG(Rei,Water_Pr,Water_k,Water_di)

print("Aci :",Aci)
print("Rei :",Rei)
print("hi :",hi)

#Pipe Outside
Aco=Across_Donut(Oil_Di,Water_do)
Dh= Oil_Di-Water_do
Reo=Reynold(Oil_m_dot/Aco,Dh,Oil_visc)
L = 357.27
hoil=hlam(Oil_Di,Water_do,Reo,Oil_Pr,Dh,L,Oil_k)
LMTD_ = LMTD(Oil_Tin,Oil_Tout,Water_Tin,Water_Tout)
U_ = Uvalue(1,hi,Water_di,1,hoil,Water_do,Material_kw)

print("Aco :",Aco)
print("Dh :",Dh)
print("Reo :",Reo)
print("hoil :",hoil)
print("LMTD_ :",LMTD_)
print("U_ :",U_)

#Check
Acheck = Qtotal/U_/LMTD_
Lcheck = Acheck/math.pi/Water_do

print("Lcheck",Lcheck)
print("Lguess",L)

Aci : 0.0013163929811405833
Rei : 36473.70623721257
hi : 2989.005809391422
Aco : 0.002585617147702373
Dh : 0.026699999999999995
Reo : 341.5590508030674
hoil : 28.71808388238451
LMTD_ : 41.78946571358157
U_ : 28.33559450909162
Lcheck 357.30482177247467
Lguess 357.27
